In [ ]:
#import
from selenium import webdriver
import time
import csv
import re
import pandas as pd

In [ ]:
#setup
abbreviations = [
    "AK", "AL", "AR", "AZ", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "IA",
    "ID", "IL", "IN", "KS", "KY", "LA", "MA", "MD", "ME", "MI", "MN", "MO",
    "MS", "MT", "NC", "ND", "NE", "NH", "NJ", "NM", "NV", "NY", "OH", "OK",
    "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VA", "VT", "WA", "WI",
    "WV", "WY","DC"
]
driver = webdriver.Chrome()
df = pd.DataFrame(columns=['Name', 'Address', 'Phone Number', 'Rating', 'URL'])
f = open('output.txt', 'w+')

In [ ]:
#main loop to go through each page filtered by state to get as much info as possible

for abr in abbreviations:
    for i in range(1,16):
        pageUrl = 'https://www.bbb.org/search?filter_state=' + abr + '&find_country=USA&find_entity=25049-000&find_id=2302_6200-3600&find_text=Payday%20Loans&find_type=Category&page=' + str(i)
        driver.get(pageUrl)
        time.sleep(5)
        text = driver.page_source
        rows = re.findall(r'<div class="card result-card">(?:.|\n)*?</div></div></div></div>', text)
        if len(rows) == 0:
            break
        for row in rows:
            url = 'null'
            name = 'null'
            address = 'null'
            phoneNumber = 'null'
            bbbRating = 'null'
            try:
                #print(row)
                url = re.findall(r'href="(?:.|\n)*?">',row)[0][6:-2]
                name = re.findall(r'<span(?:.|\n)*?</span></a></h3>',row)[0]
                name = re.findall(r'">(?:.|\n)*?</span>',name)[0][2:-7]
                bbbRating = re.findall(r'BBB Rating: (?:.|\n)*?</span>',row)[0][12:-7]
                if len(bbbRating) > 2:
                    bbbRating = bbbRating[0:2]
                phoneNumber = re.findall(r'tel:(?:.|\n)*?"',row)[0][4:-1]
                address = re.findall(r'<p class="bds-body text-size-5 text-gray-70"(?:.|\n)*?</p>',row)[0]
                lines = re.findall(r'>(?:.|\n)*?<',address)
                if len(lines) == 7:
                    line1 = lines[0][1:-1]
                    line2 = lines[3][2:-1]
                    line3 = lines[5][1:-1]
                    address = line1 + " " + line2 + " " + line3
                elif len(lines) == 4:
                    line2 = lines[0][2:-1]
                    line3 = lines[2][1:-1]
                    address = line2 + " " + line3
                elif len(lines) == 2:
                    address = lines[0][1:-1]
                else:
                    address = "no address??"
                df.loc[len(df)] = [name, address, phoneNumber, bbbRating, url]
            except:
                f.write('ERR: ' + abr + str(i) + '\n')
                f.write('Current DF Size: ' + str(len(df)) + '\n')
                print('ERR: ' + abr + str(i))
                print('Current DF Size: ' + str(len(df)))
        print(abr + str(i))
        f.write(abr + str(i) + '\n')

AK1
AL1
AL2
AL3
AL4
AL5
AL6
AL7
AL8
AL9
AL10
AL11
AL12
AL13
AL14
AL15
AR1


In [ ]:
#export scrape to csv

f.close()
df.to_csv('bbbScrape.csv')